In [38]:
from semantic_tfidf import SemanticTFIDF, SAMPLE_PARAGRAPHS as sp

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\T.PRANEETH\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Loading fasttext-wiki-news-subwords-300 word2vectors
The module load time : 186.91360020637512s


In [13]:
import os
import pandas as pd
import queue

In [25]:
def get_all_text_files(home_dir, count=10):
    q = queue.Queue()
    text_file_locs = []
    q.put(home_dir)
    file_count = 0
    while not(q.empty()):
        if file_count >= count:
            break
        dir_to_search = q.get()
        try:
            for fi in os.listdir(dir_to_search):
                loc = dir_to_search + '/' + fi
                if loc.endswith('.txt'):
                    text_file_locs.append(loc)
                    file_count += 1
                elif not(os.path.isfile(loc)):
                    q.put(loc)
                else:
                    pass
        except:
            pass
    
    your_files = pd.DataFrame({'locs':text_file_locs})
    your_files.to_csv(os.getcwd()+f'/your_files_{count}.csv')
    return your_files


my_text_files = get_all_text_files('C:/Users/T.PRANEETH', count=1000)
my_text_files.head()

,locs
0,C:/Users/T.PRANEETH/all_raw_answers.txt
1,C:/Users/T.PRANEETH/jaymama_chat.txt
2,C:/Users/T.PRANEETH/requirements.txt
3,C:/Users/T.PRANEETH/.conda/environments.txt
4,C:/Users/T.PRANEETH/anaconda3/LICENSE_PYTHON.txt


In [37]:
row = next(my_text_files.iterrows())[1]
row['locs']

'C:/Users/T.PRANEETH/all_raw_answers.txt'

In [46]:
my_text_files['locs'][0]

'C:/Users/T.PRANEETH/all_raw_answers.txt'

In [63]:
from urllib.request import urlopen
from bs4 import BeautifulSoup


def get_text_as_paras(url):
    html = urlopen(url).read()
    soup = BeautifulSoup(html, features="html.parser")

    # kill all script and style elements
    for script in soup(["script", "style"]):
        script.extract()    # rip it out

    # get text
    text = soup.get_text()

    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    text = [chunk for chunk in chunks if chunk]

    return text

In [53]:
def extract_as_paras(db_loc):
    db = pd.read_csv(db_loc)
    paras = []
    for ind in range(len(db)):
        with open(db['locs'][ind], encoding="utf8", errors="ignore")as fi:
            paras.append(fi.read())
    
    return paras

In [65]:
import time
tic = time.time()
ps = ["What is the objective of football is , The objective of football is to score more goals than the opposing team.", "How many players are on a football team is ,A football team typically consists of 11 players on the field at a time.", "What are the dimensions of a football field is ,A football field is typically 100 yards long and 50 yards wide.", "What is the maximum number of substitutions allowed in a football match is, In most football competitions, a team is allowed to make up to three substitutions per match.", "mars in 2 words is , Red planet.", "is mars fine for living is , short is Mars is not currently fine for living short term or long term because the atmosphere is too thin to provide enough protection from harmful radiation and the temperature is too cold to sustain human life without significant technological support. However, scientists are working on developing technology and infrastructure that could potentially make it possible for humans to live on Mars in the future."]
paras = get_text_as_paras('https://www.ibm.com/in-en/topics/deep-learning#:~:text=Machine%20Learning%20Accelerator-,What%20is%20deep%20learning%3F,from%20large%20amounts%20of%20data.')
doc_finder = SemanticTFIDF(paras)
tac = time.time()
print(tac-tic)

0.740365743637085


In [66]:
paras

['What is Deep Learning? | IBM',
 'Deep Learning',
 'Deep Learning',
 'Deep learning attempts to mimic the human brain—albeit far from matching its ability—enabling systems to cluster data and make predictions with incredible accuracy.',
 'Featured products',
 'Watson Studio',
 'Watson Machine Learning Accelerator',
 'What is deep learning?',
 'Deep learning is a subset of\xa0machine learning, which is essentially a neural network with three or more layers. These neural networks attempt to simulate the behavior of the human brain—albeit far from matching its ability—allowing it to “learn” from large amounts of data. While a neural network with a single layer can still make approximate predictions, additional hidden layers can help to optimize and refine for accuracy.',
 'Deep learning drives many\xa0artificial intelligence (AI)\xa0applications and services that improve automation, performing analytical and physical tasks without human intervention. Deep learning technology lies behind 

In [85]:
doc_finder("application technology")

The given query word application best suits application
The given query word technology best suits technology


('deep learning drive many artificial intelligence ( ai ) application service improve automation , performing analytical physical task without human intervention . deep learning technology lie behind everyday product service ( such digital assistants , voice-enabled tv remotes , credit card fraud detection ) well emerging technology ( such self-driving cars ) .',
 9)

In [18]:
fi = open(r'C:\Users\T.PRANEETH\jaymama_chat.txt', encoding="utf8", errors="ignore")
chat_lines = fi.readlines()

In [28]:
chat_lines = list(map(str, chat_lines))
tic = time.time()
chat_finder = SemanticTFIDF(chat_lines)
tac = time.time()
print(tac-tic)

30.681381464004517


In [107]:
chat_finder("anonymous logging usage")

The given query word anonymous best suits anonymous
The given query word logging best suits logged
The given query word usage best suits use


"b ' [ 28/07/20 , 09:19:57 ] pranu : jay mama , yesterday u said abt anonymous logins babaye\\xe2\\x80\\x99s account , intimated friend soft tone double check entering netflix profile , replied never enter profile also personally keep check viewing history regularly . sure guy guess could pranu\\xe2\\x80\\x99s ( pranay\\xe2\\x80\\x99s ) friends . guess\\r\\n '"

0.025